This is The Notebook for the Assignment for Day 4

In [88]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

DATA = Path("/Users/khaledalamro/Desktop/SDAIA_bootcamp/bootcamp_week2/week2-data-work/data/processed/orders_realistic_100_rows.parquet")
FIGS = Path("/Users/khaledalamro/Desktop/SDAIA_bootcamp/bootcamp_week2/week2-data-work/reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

In [89]:
df = pd.read_parquet(DATA)
print(len(df),"\n******************")
print(df.dtypes.head(15),"\n******************")
print(df.isna().sum().sort_values(ascending=False).head(10))

100 
******************
order_id           object
user_id            object
amount            float64
quantity          float64
created_at          int64
status             object
status_clean       object
amount__isna         bool
quantity__isna       bool
date               object
year                int64
month              object
dow                object
hour                int64
country            object
dtype: object 
******************
amount           13
amount_winsor    13
quantity          5
order_id          0
year              0
signup_date       0
country           0
hour              0
dow               0
month             0
dtype: int64


# The Dataset has some missing values but i think its not that bad 


In [90]:
# Question 1: Who is the most user has profit to the company in sales?
    
df['total_sales_value'] = df['amount'] * df['quantity']

q1_table = df.groupby('user_id')['total_sales_value'].sum().reset_index()
q1_table = q1_table.sort_values('total_sales_value', ascending=False)

print("Top 5 Users by Sales (Amount * Quantity):")
display(q1_table.head())

fig1 = px.bar(
    q1_table.head(10), 
    x='user_id', 
    y='total_sales_value',
    title='Total Sales Revenue by User (Amount × Quantity)',
    labels={'total_sales_value': 'Total Sales ($)', 'user_id': 'User ID'}
)
fig1.show()
save_fig(fig1, FIGS / "Q1_user_Sales.png")


Top 5 Users by Sales (Amount * Quantity):


,user_id,total_sales_value
33,0040,2831.00
0,0001,2577.11
30,0036,2503.97
25,0031,2464.84
41,0048,1978.25


* User 0040 is the clear top in sales $2,831 in total.

* Since we are calculating Amount × Quantity, these users likely made multiple purchases of high-quantity items, differentiating them from users who just bought one expensive item.

In [91]:
# 2- What is the average time gap between user signup and his first order?

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['signup_date'] = pd.to_datetime(df['signup_date'], errors='coerce')

first_orders = df.dropna(subset=['date']).groupby('user_id')['date'].min().reset_index()
user_signups = df[['user_id', 'signup_date']].drop_duplicates()

q2_table = pd.merge(first_orders, user_signups, on='user_id')
q2_table['days_gap'] = (q2_table['date'] - q2_table['signup_date']).dt.days

display(q2_table[['user_id', 'days_gap']].head())
print(f"Average Gap: {q2_table['days_gap'].mean():.2f} days")

fig2 = px.histogram(
    q2_table, 
    x='days_gap', 
    nbins=10,
    title='Distribution of Days from Signup to First Order',
    labels={'days_gap': 'Days (Signup to Order)'}
)
fig2.show()
save_fig(fig2, FIGS / "Q2_signup_gap.png")


,user_id,days_gap
0,0001,22
1,0003,45
2,0004,5
3,0005,2
4,0006,14


Average Gap: 15.52 days


* The Average user takes about 2 weeks to place their first order after signing up.

* The minimum gap is 1 day, proving that some users purchase after signing up.

In [92]:
# 3- What are the peak hours of the day for ordering?

q3_table = df['hour'].dropna().value_counts().reset_index()
q3_table.columns = ['hour', 'order_count']
q3_table = q3_table.sort_values('hour')
display(q3_table)

fig3 = px.line(
    q3_table, 
    x='hour', 
    y='order_count',
    title='Order Volume by Hour of Day',
    labels={'hour': 'Hour (0-23)', 'order_count': 'Number of Orders'}
)
fig3.update_xaxes(dtick=1) 
fig3.show()
save_fig(fig3, FIGS / "Q3_peak_hours.png")


,hour,order_count
16,0,3
20,1,2
2,2,7
13,3,4
10,4,4
15,5,3
22,6,1
19,7,2
9,8,5
4,9,6


* The peak is at 13:00 (1 PM) with 10 orders. This might be because the work break.

* The second strong spike at 20:00 (8 PM), browsing  after work hours.

* if there is any marketing notifications it should be around 12:30 PM or 7:30 to be where the is alot of active users

In [93]:
# 4- Which countries are in the top in sales?

unique_users = df[['user_id', 'country']].drop_duplicates()
q4_table = unique_users['country'].value_counts().reset_index()
q4_table.columns = ['country', 'user_count']
display(q4_table)

fig4 = px.pie(
    q4_table, 
    values='user_count', 
    names='country',
    title='Distribution of Users by Country',
)

fig4.update_traces(textinfo='percent+value')
fig4.show()
save_fig(fig4, FIGS / "Q4_user_locations_pie.png")


,country,user_count
0,SA,34
1,AE,32


* its very simple graph the users is spliting into to countries SA and UAE

In [94]:
d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))

group_sa = d.loc[d["country"].eq("SA"), "is_refund"]
group_ae = d.loc[d["country"].eq("AE"), "is_refund"]

print(f"Sample Size SA: {len(group_sa)}")
print(f"Sample Size AE: {len(group_ae)}")
print("-" * 30)

def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 42) -> dict:
    rng = np.random.default_rng(seed)
    a_vals = a.dropna().to_numpy()
    b_vals = b.dropna().to_numpy()
    
    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a_vals, size=len(a_vals), replace=True)
        sb = rng.choice(b_vals, size=len(b_vals), replace=True)
        diffs.append(sa.mean() - sb.mean())
    
    diffs = np.array(diffs)
    
    return {
        "diff_mean": float(a_vals.mean() - b_vals.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }

result = bootstrap_diff_means(group_sa, group_ae)

print(f"Observed Difference (SA - AE): {result['diff_mean']:.4f}")
print(f"95% CI: [{result['ci_low']:.4f}, {result['ci_high']:.4f}]")

if result['ci_low'] <= 0 <= result['ci_high']:
    print("CONCLUSION: The difference is NOT statistically significant (CI contains 0).")
else:
    print("CONCLUSION: The difference IS statistically significant (CI excludes 0).")

Sample Size SA: 50
Sample Size AE: 50
------------------------------
Observed Difference (SA - AE): -0.0400
95% CI: [-0.2000, 0.1200]
CONCLUSION: The difference is NOT statistically significant (CI contains 0).
